In [1]:
import cv2
import os
import numpy as np
from google.colab.patches import cv2_imshow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.getcwd()

'/content'

In [ ]:
#os.chdir('/content/drive/My Drive/Colab Notebooks/yolov5')
os.chdir('/content/drive/MyDrive/ColabINF634/yolov5')
os.getcwd()

'/content/drive/.shortcut-targets-by-id/1x_XyufWNiEZ7trNaHNqvZ_n2ZDUaxNFw/Colab Notebooks/yolov5'

In [ ]:
pip install ffmpeg-python

In [ ]:
 import ffmpeg    

 def check_rotation(path_video_file):
     # this returns meta-data of the video file in form of a dictionary
     meta_dict = ffmpeg.probe(path_video_file)
     print(meta_dict['streams'][0]['tags'])
     # from the dictionary, meta_dict['streams'][0]['tags']['rotate'] is the key
     # we are looking for
     rotateCode = None
     if int(meta_dict['streams'][0]['tags']['rotate']) == 90:
         rotateCode = cv2.ROTATE_90_CLOCKWISE
     elif int(meta_dict['streams'][0]['tags']['rotate']) == 180:
         rotateCode = cv2.ROTATE_180
     elif int(meta_dict['streams'][0]['tags']['rotate']) == 270:
         rotateCode = cv2.ROTATE_90_COUNTERCLOCKWISE

     return rotateCode

 def correct_rotation(frame, rotateCode):  
     return cv2.rotate(frame, rotateCode) 
     

In [ ]:
# Fonction pour lire + save les frames de la videos dans repertoire

def read_save_frames(videoname, rotate=False):
  path = os.getcwd()+'/data/videos/'+videoname[:-4]+'_frames'
  if not os.path.exists(path):
    os.mkdir(path)

  if rotate == True: 
    rotateCode = check_rotation('data/videos/'+videoname)

  cap = cv2.VideoCapture('data/videos/'+videoname)
  if (cap.isOpened()== False): 
    print("Error opening video stream or file")

  if cap.isOpened():
    i = 0
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
  while (cap.isOpened()):
    ret, frame = cap.read()
    if ret:  
      if rotate == True:
        if rotateCode is not None:
          frame = correct_rotation(frame, rotateCode)
      path2 = os.getcwd()+'/data/videos/'+videoname[:-4]+'_frames/frame_'+str(i).zfill(4)+'.jpg'
      #print(path)
      status = cv2.imwrite(path2, frame)
      #print(status)
      i += 1
    # Break the loop
    else: 
        break
  # When everything done, release the video capture object
  cap.release()
  # Closes all the frames
  cv2.destroyAllWindows()


In [ ]:
#Create a simple list of files
def get_file_list(path_people,extension):
  filenames=[]
  for item in os.listdir(path_people):
    if item[-4:]==extension:
      filenames.append(item)
  return filenames

#Create a dico with the content of the mask BB
def create_dic(path_masks,filenames):
  dic={}
  for image in filenames:
    data=[]
    try:
      file=open(path_masks+'/labels/'+image[:-4]+'.txt',"r")
      lines=file.readlines()
      file.close
      for line in lines:
        val=np.array(line.split())
        data.append(val)
      file.close()
      data=np.array(data)
      dic[image[:-4]]=data
    except:
      dic[image[:-4]]=np.zeros((0,6))
  return dic

def score_rattachement(col_min,col_max,row_min,row_max,col_min2,col_max2,row_min2,row_max2,confidence): #BBpeople puis BBmask
  if (col_min2>=col_min and col_max2<=col_max and row_min2>=row_min and row_max2<=row_max):
    #print(col_min,col_max,row_min,row_max,col_min2,col_max2,row_min2,row_max2,1)
    return confidence
  else: 
    #Compute la part de la BBmask qui est contenue dans BBpeople
    #print(col_min,col_max,row_min,row_max,col_min2,col_max2,row_min2,row_max2)
    col_min_max=max(col_min,col_min2)
    col_max_min=min(col_max,col_max2)
    row_min_max=max(row_min,row_min2)
    row_max_min=min(row_max,row_max2)
    if (row_max_min>=row_min_max and col_max_min>col_min_max):
      reponse1 = (row_max_min-row_min_max)/(row_max2-row_min2)
      reponse2 = (col_max_min-col_min_max)/(col_max2-col_min2)
      ##print(col_min_max,col_max_min,row_min_max,row_max_min)
      ##print("reponse1 "+str(reponse1))
      ##print("reponse2 "+str(reponse2))
      ##print(col_min,col_max,row_min,row_max,col_min2,col_max2,row_min2,row_max2,reponse1,reponse2,reponse1*reponse2)
      return reponse1*reponse2*confidence
    ##print(col_min,col_max,row_min,row_max,col_min2,col_max2,row_min2,row_max2,0)
    return 0

#Et maintenant le corps du code qui fait le boulot
#Version 2 de la fonction, celle qui met aussi la BB du mask dans le résultat
def merge_yolos(path_people,path_masks,path_output,filenames):
  dic = create_dic(path_masks,filenames)
  for i in range(len(filenames)): #i pour chaque image
    try:
      file=open(path_people+'/labels/'+filenames[i][:-4]+'.txt',"r")
      lines=file.readlines()
      file.close
      file2=open(path_output+filenames[i][:-4]+'.txt',"w")
      for line in lines: # Chaque ligne = 1 people
        val=np.array(line.split()) #Ce qui nous intéresse c'est val[1:5] qui correspond à la BB
        col_min=float(val[1])-float(val[3])/2
        col_max=float(val[1])+float(val[3])/2
        row_min=float(val[2])-float(val[4])/2
        row_max=float(val[2])+float(val[4])/2
  
        # Mon critère (on pourra améliorer): je reconnais un mask/no_mask comme appartenant à un people s'il est contenu dans la BB du people
        line2=''
        score_max=0
        candidat_garde=2 #Si on ne trouve pas de candidat, on mettra 2.
        best_candidat=0
        for candidat in dic[filenames[i][:-4]]:
          col_min2=float(candidat[1])-float(candidat[3])/2
          col_max2=float(candidat[1])+float(candidat[3])/2
          row_min2=float(candidat[2])-float(candidat[4])/2
          row_max2=float(candidat[2])+float(candidat[4])/2
          confidence=float(candidat[5])
          score = score_rattachement(col_min,col_max,row_min,row_max,col_min2,col_max2,row_min2,row_max2,confidence)
          if score>score_max:
            score_max=score
            candidat_garde=candidat
        if candidat_garde==2:
          file2.write(line[:-1]+" "+str(candidat_garde)+line[1:-1]+'\n') #On n'a pas trouve de candidat : juste un 2 puis la même BB 
        else:
          file2.write(line[:-1]+" "+str(candidat_garde[0])+' '+str(candidat_garde[1])+' '+str(candidat_garde[2])+' '+str(candidat_garde[3])+' '+str(candidat_garde[4])+'\n') #On garde la BB du mask dont on va se servir pour l'estimation des distances
      file2.close()
      file.close()
      print("Yolos merged for "+filenames[i])
    except:
      print("No people in "+filenames[i])

In [ ]:
# Function to help the vizualisation : Une seule image à chaque fois qui montre la prédiction fusionnée.
import matplotlib.pyplot as plt
import matplotlib.patches as patches
def show_image_and_mask2(path_input,input_image,path_bb,path_output, output_image):
    image=plt.imread(os.path.join(path_input+"/"+input_image))

    width=image.data.shape[1]
    height=image.data.shape[0]
    fig,ax=plt.subplots(1)
    ax.axis("off")
    fig.set_size_inches(10,5)

    temp=[]
    try:
      #print(path_output+input_image[:-4]+'.txt')
      file=open(path_bb+input_image[:-4]+'.txt',"r")
      lines=file.readlines()
      file.close
      for line in lines:
        val=np.array(line.split())
        temp.append(val)
      file.close()
      temp=np.array(temp)
    except:
      print("No output data for "+ image[:-4])
    for i in range(len(temp)):
      a=int( ( float(temp[i][1]) - float(temp[i][3])/2)*width)
      b=int((float(temp[i][2])-float(temp[i][4])/2)*height)
      c=int(float(temp[i][3])*width)
      d=int(float(temp[i][4])*height)
      #print(temp[i][5:])
      if temp[i][6]=="0":
        patch=patches.Rectangle((a,b),c,d,linewidth=2, edgecolor='g',facecolor="none",)
        ax.add_patch(patch)
      if temp[i][6]=="1":
        patch=patches.Rectangle((a,b),c,d,linewidth=2, edgecolor='r',facecolor="none",)
        ax.add_patch(patch)
      if temp[i][6]=="2":
        patch=patches.Rectangle((a,b),c,d,linewidth=2, edgecolor='y',facecolor="none",)
        ax.add_patch(patch)
    ax.imshow(image)  
    fig.savefig(os.path.join(path_output+"/"+output_image),bbox_inches='tight')
    plt.close()

def cree_repertoire_si_existe_pas(path):
  try:
    os.mkdir(path)
    print('Path créé : '+path)
  except:
    print('Path déjà existant (ou impossible à créer car la racine elle-même nexiste pas): '+path)

In [ ]:
#------------ MAIN CODE ------------------------
# Eulalie : it is really all your code, I just corrected a few details and decomposed in steps to help understand
# STEP 1 : PREPARE THE DIFFERENT PATHS
videoname = 'video2.MOV'
current_path=os.getcwd()
path_input=current_path+'/data/videos/'+videoname[:-4]+'_frames'
path_people=current_path+'/runs/detect/video2_people2'
path_masks=current_path+'/runs/detect/video2_mask2'
path_output=path_people+'/labels_complete/'

In [ ]:
#STEP 2 : LOAD THE VIDEO AND DECOMPOSE IT INTO FRAMES. STORE IN ./data/videos/
read_save_frames(videoname, rotate = True)

{'rotate': '90', 'creation_time': '2020-12-04T13:21:51.000000Z', 'language': 'und', 'handler_name': 'Core Media Data Handler', 'encoder': "'avc1'"}


In [ ]:
#STEP 3 - Detect people in every frame - STORE IN ./runs/detect/video_cedric_people (en ajoutant un nouveau numéro de version au besoin)
!python detect.py --weights weights/yolov5s.pt --img 640 --conf 0.25 --source data/videos/video2_frames/ --save-txt --save-conf --name video2_people --class 0

Namespace(agnostic_nms=False, augment=False, classes=[0], conf_thres=0.25, device='', exist_ok=False, img_size=640, iou_thres=0.45, name='video2_people', project='runs/detect', save_conf=True, save_txt=True, source='data/videos/video2_frames/', update=False, view_img=False, weights=['weights/yolov5s.pt'])
Using torch 1.7.0+cu101 CPU

Fusing layers... 
Model Summary: 232 layers, 7459581 parameters, 0 gradients
image 1/442 /content/drive/.shortcut-targets-by-id/1x_XyufWNiEZ7trNaHNqvZ_n2ZDUaxNFw/Colab Notebooks/yolov5/data/videos/video2_frames/frame_0000.jpg: 480x640 7 persons, Done. (0.530s)
image 2/442 /content/drive/.shortcut-targets-by-id/1x_XyufWNiEZ7trNaHNqvZ_n2ZDUaxNFw/Colab Notebooks/yolov5/data/videos/video2_frames/frame_0001.jpg: 480x640 7 persons, Done. (0.389s)
image 3/442 /content/drive/.shortcut-targets-by-id/1x_XyufWNiEZ7trNaHNqvZ_n2ZDUaxNFw/Colab Notebooks/yolov5/data/videos/video2_frames/frame_0002.jpg: 480x640 7 persons, Done. (0.393s)
image 4/442 /content/drive/.shortcu

In [ ]:
# STEP 4 - Detect mask in frames - STORE IN ./runs/detect/video_cedric_mask (en ajoutant un nouveau numéro de version au besoin)
!python detect.py --weights runs/train/mask_yolov5s_results6/weights/last.pt --img 640 --conf 0.25 --source data/videos/video2_frames/ --save-txt --save-conf --name video2_mask

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, img_size=640, iou_thres=0.45, name='video2_mask', project='runs/detect', save_conf=True, save_txt=True, source='data/videos/video2_frames/', update=False, view_img=False, weights=['runs/train/mask_yolov5s_results6/weights/last.pt'])
Using torch 1.7.0+cu101 CPU

Fusing layers... 
Model Summary: 232 layers, 7249215 parameters, 0 gradients
image 1/442 /content/drive/.shortcut-targets-by-id/1x_XyufWNiEZ7trNaHNqvZ_n2ZDUaxNFw/Colab Notebooks/yolov5/data/videos/video2_frames/frame_0000.jpg: 480x640 6 masks, 1 no-masks, Done. (0.390s)
image 2/442 /content/drive/.shortcut-targets-by-id/1x_XyufWNiEZ7trNaHNqvZ_n2ZDUaxNFw/Colab Notebooks/yolov5/data/videos/video2_frames/frame_0001.jpg: 480x640 6 masks, 1 no-masks, Done. (0.377s)
image 3/442 /content/drive/.shortcut-targets-by-id/1x_XyufWNiEZ7trNaHNqvZ_n2ZDUaxNFw/Colab Notebooks/yolov5/data/videos/video2_frames/frame_0002.jpg: 480x640 6 masks, 1 n

In [ ]:
# STEP 5 - Merge the two detection - STORE IN ./runs/detect/video_cedric_people(éventuellement video_cedric_people2 etc)/labels_complete/
cree_repertoire_si_existe_pas(path_output)
filenames=get_file_list(path_people,".jpg") #List of all the frames names
merge_yolos(path_people,path_masks,path_output,filenames) #Anciennement fait_le_travail mais c'est plus explicite comme cela

Path déjà existant (ou impossible à créer car la racine elle-même nexiste pas): /content/drive/.shortcut-targets-by-id/1x_XyufWNiEZ7trNaHNqvZ_n2ZDUaxNFw/Colab Notebooks/yolov5/runs/detect/video2_people2/labels_complete/


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:82: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


Yolos merged for frame_0000.jpg
Yolos merged for frame_0001.jpg
Yolos merged for frame_0002.jpg
Yolos merged for frame_0003.jpg
Yolos merged for frame_0004.jpg
Yolos merged for frame_0005.jpg
Yolos merged for frame_0006.jpg
Yolos merged for frame_0007.jpg
Yolos merged for frame_0008.jpg
Yolos merged for frame_0009.jpg
Yolos merged for frame_0010.jpg
Yolos merged for frame_0011.jpg
Yolos merged for frame_0012.jpg
Yolos merged for frame_0013.jpg
Yolos merged for frame_0014.jpg
Yolos merged for frame_0015.jpg
Yolos merged for frame_0016.jpg
Yolos merged for frame_0017.jpg
Yolos merged for frame_0018.jpg
Yolos merged for frame_0019.jpg
Yolos merged for frame_0020.jpg
Yolos merged for frame_0021.jpg
Yolos merged for frame_0022.jpg
Yolos merged for frame_0023.jpg
Yolos merged for frame_0024.jpg
Yolos merged for frame_0025.jpg
Yolos merged for frame_0026.jpg
Yolos merged for frame_0027.jpg
Yolos merged for frame_0028.jpg
Yolos merged for frame_0029.jpg
Yolos merged for frame_0030.jpg
Yolos me

In [ ]:
# STEP 6 - SAVE RESULTING FRAMES FOR VISUALISATION IN ./runs/detect/video_cedric_people(éventuellement video_cedric_people2 etc)/labels_complete/results
# Save images
i=0
path_output_bis = path_output+'results'
cree_repertoire_si_existe_pas(path_output_bis)
for filename in filenames:
  show_image_and_mask2(path_input,filename,path_output,path_output_bis,filename)
  i+=1

# And make a videoout of them
w, h = cv2.imread(path_output_bis+'/'+filenames[0]).shape[0:2]
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(path_output_bis+'/final.mp4',fourcc, 20.0, (h,w))
for filename in filenames:
  frame = cv2.imread(path_output_bis+'/'+filename)
  out.write(frame)
out.release()

Path déjà existant (ou impossible à créer car la racine elle-même nexiste pas): /content/drive/.shortcut-targets-by-id/1x_XyufWNiEZ7trNaHNqvZ_n2ZDUaxNFw/Colab Notebooks/yolov5/runs/detect/video2_people2/labels_complete/results


In [ ]:
os.getcwd()

'/content/drive/.shortcut-targets-by-id/1x_XyufWNiEZ7trNaHNqvZ_n2ZDUaxNFw/Colab Notebooks/yolov5'

In [ ]:
os.chdir('..')

In [ ]:
os.getcwd()

'/content/drive/.shortcut-targets-by-id/1x_XyufWNiEZ7trNaHNqvZ_n2ZDUaxNFw/Colab Notebooks'

In [ ]:
%pip install filterpy
from filterpy.kalman import KalmanFilter

In [ ]:
from sort import *

In [ ]:
os.chdir('./yolov5')

In [ ]:
#---- TRACKER
output_path_tracker_txt = '/content/drive/MyDrive/ColabINF634/yolov5/runs/detect/video2_people2/labels_complete/results_eulalie'
output_path_tracker_frames = '/content/drive/MyDrive/ColabINF634/yolov5/runs/detect/video2_people2/labels_complete/results_eulalie/frames'


#'/content/drive/My Drive/Colab Notebooks/yolov5/runs/detect/video_cedric_people3/labels_complete/results_eulalie_test'
cree_repertoire_si_existe_pas(output_path_tracker_txt)
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
filenames=sorted(get_file_list(path_people,".jpg"))
classes = ['mask', 'no-mask', 'unknown']
colors = [(0, 255, 0), (255, 0, 0), (255, 128, 0)]
mot_tracker = Sort(max_age = 20) #max age c'est le nb de frames avant de suppr un tracker 

ids_appearance = {}
for input_image in filenames:
    #print(input_image)
    # Open file with final detection bounding boxes 
    file=open(path_output+'/'+input_image[:-4]+'.txt',"r")
    lines=file.readlines()
    file.close
    temp = []
    temp2 = []
    count = 0

    for line in lines:
      count += 1 
      val=line.split()
      val = [float(v) for v in val]
      val2 = val[1:5]
      val2.append(count)
      val2.append(val[6]) # A affiner pour garder la classe
      val2.append(val[6])

      temp.append(np.array(val2))

      #on fait la meme chose pr les bb masques: 
      temp2.append(val[7:11])

    detections = np.array(temp)
    mask_bbs = np.array(temp2)

    img=plt.imread(os.path.join(path_input+"/"+input_image))
    width=img.data.shape[1]
    height=img.data.shape[0]

    # Transform to torch tensor
    detections = torch.from_numpy(detections).float().to(device)
    detections[:,0] = (detections[:,0]- detections[:,2]/2) * width
    detections[:,1] = (detections[:,1]- detections[:,3]/2) * height
    detections[:,2] = detections[:,0] + detections[:,2]*width
    detections[:,3] = detections[:,1] + detections[:,3]*height
    
    mask_bbs = torch.from_numpy(mask_bbs).float().to(device)
    mask_bbs[:,0] = (mask_bbs[:,0]- mask_bbs[:,2]/2) * width
    mask_bbs[:,1] = (mask_bbs[:,1]- mask_bbs[:,3]/2) * height
    mask_bbs[:,2] = mask_bbs[:,0] + mask_bbs[:,2]*width
    mask_bbs[:,3] = mask_bbs[:,1] + mask_bbs[:,3]*height

    # Create file 
    file2 =open(output_path_tracker_txt+'/'+input_image[:-4]+'.txt',"w")
    if detections is not None:
        tracked_objects = mot_tracker.update(detections.cpu(), mask_bbs)
        unique_labels = detections[:, -1].cpu().unique()
        n_cls_preds = len(unique_labels)
        for x1, y1, x2, y2, obj_id, cls_pred, cls_original, mask_bb_x1, mask_bb_y1, mask_bb_x2, mask_bb_y2 in tracked_objects:
            if np.isnan(y1):
              print('nan')
              continue
            # Dictionary of appearences and cls returned by merge yolo (!!Not by tracker!!) 
            if obj_id not in ids_appearance.keys():
              ids_appearance[obj_id] = {}
              ids_appearance[obj_id]['frames'] =  []
              ids_appearance[obj_id]['cls'] = []
            ids_appearance[obj_id]['frames'].append(input_image)
            ids_appearance[obj_id]['cls'].append(cls_original)
            # Save info we need in txt file
            file2.write(str(obj_id)+" "+str(x1)+" "+str(y1)+" "+str(x2)+" "+str(y2)+" "+str(cls_pred)+" "+
                        str(cls_original)+" "+str(mask_bb_x1)+" "+str(mask_bb_y1)+" "+str(mask_bb_x2)+" "+str(mask_bb_y2)+" "+'\n')
  

    file2.close()
          
    print("Done : "+output_path_tracker_txt+'/'+input_image)



Path déjà existant (ou impossible à créer car la racine elle-même nexiste pas): /content/drive/MyDrive/ColabINF634/yolov5/runs/detect/video2_people2/labels_complete/results_eulalie
Done : /content/drive/MyDrive/ColabINF634/yolov5/runs/detect/video2_people2/labels_complete/results_eulalie/frame_0000.jpg


/content/drive/.shortcut-targets-by-id/1x_XyufWNiEZ7trNaHNqvZ_n2ZDUaxNFw/Colab Notebooks/sort.py:33: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "iou" failed type inference due to: non-precise type pyobject
[1] During: typing of argument at /content/drive/.shortcut-targets-by-id/1x_XyufWNiEZ7trNaHNqvZ_n2ZDUaxNFw/Colab Notebooks/sort.py (38)

File "../sort.py", line 38:
def iou(bb_test,bb_gt):
    <source elided>
  """
  xx1 = np.maximum(bb_test[0], bb_gt[0])
  ^

  @jit
/usr/local/lib/python3.6/dist-packages/numba/object_mode_passes.py:178: NumbaWarning: Function "iou" was compiled in object mode without forceobj=True.

File "../sort.py", line 34:
@jit
def iou(bb_test,bb_gt):
^

  state.func_ir.loc))
/usr/local/lib/python3.6/dist-packages/numba/object_mode_passes.py:188: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For m

Done : /content/drive/MyDrive/ColabINF634/yolov5/runs/detect/video2_people2/labels_complete/results_eulalie/frame_0001.jpg
Done : /content/drive/MyDrive/ColabINF634/yolov5/runs/detect/video2_people2/labels_complete/results_eulalie/frame_0002.jpg
Done : /content/drive/MyDrive/ColabINF634/yolov5/runs/detect/video2_people2/labels_complete/results_eulalie/frame_0003.jpg
Done : /content/drive/MyDrive/ColabINF634/yolov5/runs/detect/video2_people2/labels_complete/results_eulalie/frame_0004.jpg
Done : /content/drive/MyDrive/ColabINF634/yolov5/runs/detect/video2_people2/labels_complete/results_eulalie/frame_0005.jpg
Done : /content/drive/MyDrive/ColabINF634/yolov5/runs/detect/video2_people2/labels_complete/results_eulalie/frame_0006.jpg
Done : /content/drive/MyDrive/ColabINF634/yolov5/runs/detect/video2_people2/labels_complete/results_eulalie/frame_0007.jpg
Done : /content/drive/MyDrive/ColabINF634/yolov5/runs/detect/video2_people2/labels_complete/results_eulalie/frame_0008.jpg
Done : /content/

In [ ]:
#------ TRACKER POST-PROCESSING
for key in ids_appearance.keys():
  classes = ids_appearance[key]['cls']
  C0 = classes.count(0.0)
  C1 = classes.count(1.0)
  C2 = classes.count(2.0)
  seuil = 5
  if C2 != 0 and (C1 > seuil or C0 > seuil): 
    if C1 >= C0: 
      maj = 1.0
    else:
      maj = 0.0 

    for i in range(len(classes)): 
      ids_appearance[key]['cls'][i] = maj
  if C2 != 0 and C1 < seuil and C0 < seuil: 
    for i in range(len(classes)): 
      ids_appearance[key]['cls'][i] = 2.0

  #if 2.0 in classes and 1.0 in classes: 
  #  for i in range(len(classes)): 
  #    ids_appearance[key]['cls'][i] = 1.0
  #elif 2.0 in classes and 0.0 in classes: 
  #  for i in range(len(classes)): 
  #    ids_appearance[key]['cls'][i] = 0.0
  #elif 1.0 in classes and 0.0 in classes: 
  #  for i in range(len(classes)): 
  #    ids_appearance[key]['cls'][i] = 2.0 

# Now we have appropriate classes for all frames so we go to change the .txt file

for key in ids_appearance.keys():
  for i,frame in enumerate(ids_appearance[key]['frames']):
    file = open(output_path_tracker_txt+'/'+frame[:-4]+'.txt',"r")
    lines=file.readlines()
    file.close()
    file2 = open(output_path_tracker_txt+'/'+frame[:-4]+'.txt',"w")
    if lines!=[]:
      for i,line in enumerate(lines):
        val = line.split()
        val = [float(v) for v in val]
        #print(val)
        if val[0] == key:
          val[5] = ids_appearance[key]['cls'][0]
          val = [str(v) for v in val]
          line = ' '.join(val)
          lines[i] = line+'\n'
      for line in lines:
        file2.write(line)
      file2.close()
    
print("Done")




Done


In [ ]:
cree_repertoire_si_existe_pas(output_path_tracker_frames)

classes = ['mask', 'no-mask', 'unknown']


for input_image in filenames: 
  img=plt.imread(os.path.join(path_input+"/"+input_image))
  file = open(output_path_tracker_txt+'/'+input_image[:-4]+'.txt',"r")
  lines = file.readlines()
  file.close()
  for line in lines: 
    bbox=line.split()
    bbox = [float(v) for v in bbox]    
    #bbox = np.array(bbox[:6])
    obj_id = int(bbox[0])
    x1 = float(bbox[1])
    y1 = float(bbox[2])
    x2 = float(bbox[3])
    y2 = float(bbox[4])
    cls_pred = int(bbox[5])

    box_h = int((y2 - y1)) 
    box_w = int((x2 - x1)) 
    y1 = int(y1) 
    x1 = int(x1) 
            #color = colors[int(obj_id) % len(colors)]
    cls = classes[int(cls_pred)]
    color = colors[int(cls_pred)]

    img = cv2.rectangle(img, (x1, y1), (x1+box_w, y1+box_h), color, 4)
    img = cv2.rectangle(img, (x1, y1-35), (x1+len(cls)*19+80, y1), color, -1)
    img = cv2.putText(img, cls + "-" + str(int(obj_id)), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 3)
    #recolor correctly the frame
    #print(img.shape)
  img = img[:, :, [2, 1, 0]]
  cv2.imwrite(output_path_tracker_frames+'/'+input_image, img)
  print("Done : "+output_path_tracker_frames+'/'+input_image)

Path déjà existant (ou impossible à créer car la racine elle-même nexiste pas): /content/drive/MyDrive/ColabINF634/yolov5/runs/detect/video2_people2/labels_complete/results_eulalie/frames
Done : /content/drive/MyDrive/ColabINF634/yolov5/runs/detect/video2_people2/labels_complete/results_eulalie/frames/frame_0000.jpg
Done : /content/drive/MyDrive/ColabINF634/yolov5/runs/detect/video2_people2/labels_complete/results_eulalie/frames/frame_0001.jpg
Done : /content/drive/MyDrive/ColabINF634/yolov5/runs/detect/video2_people2/labels_complete/results_eulalie/frames/frame_0002.jpg
Done : /content/drive/MyDrive/ColabINF634/yolov5/runs/detect/video2_people2/labels_complete/results_eulalie/frames/frame_0003.jpg
Done : /content/drive/MyDrive/ColabINF634/yolov5/runs/detect/video2_people2/labels_complete/results_eulalie/frames/frame_0004.jpg
Done : /content/drive/MyDrive/ColabINF634/yolov5/runs/detect/video2_people2/labels_complete/results_eulalie/frames/frame_0005.jpg
Done : /content/drive/MyDrive/Co

In [ ]:
#Video finale
## SAVE VIDEO 
w, h = cv2.imread(output_path_tracker_frames+'/'+filenames[0]).shape[0:2]
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path_tracker_frames+'/final_tracker.mp4',fourcc, 20.0, (h,w))

for filename in filenames:
  frame = cv2.imread(output_path_tracker_frames+'/'+filename)
  
  out.write(frame)

out.release()
cv2.destroyAllWindows()